In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [2]:
"""
Rohlik Group ML Engineer Assignment

Goal:
  Objective 2 (Forecasting): predict next 7 days' sales per product assuming sell_price stays at last day.
  Objective 1 (Optimal Pricing): choose sell_price per product per day to maximize 7-day revenue
    subject to total 7-day profit >= 7 * (profit from last day).

This script intentionally implements:
  - 3 forecasting approaches (baseline + 2 modern GBDT models)
  - 3 demand models for price-response (elasticity + monotonic GBDTs)
  - 3 pricing optimizers (constant, greedy repair, lagrangian) and selects the best feasible

Why not only "foundation models"?
  In 2024–2025, time-series foundation models like TimesFM / TimeGPT / Chronos became popular,
  but they add external dependencies, downloads, or API keys. In interview take-homes, you want
  code that runs cleanly in a reviewer environment. So we:
    - deliver strong, production-grade tabular forecasting (CatBoost/LightGBM)
    - include optional hooks/commentary for foundation models (disabled by default)
"""

import numpy as np
import pandas as pd
from datetime import timedelta
from dataclasses import dataclass

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
from catboost import CatBoostRegressor


# -----------------------------
# Config
# -----------------------------
DATA_PATH = "ml_task_data.csv"  # update if needed
HORIZON_DAYS = 7

# Feature settings for ML forecasters
LAGS = [1, 7]
ROLL_WINDOWS = [7, 14]

# Pricing candidate controls
MAX_PRICE_CANDIDATES = 60
PRICE_UPLIFT_CAP = 1.20  # allow up to +20% beyond historical max to avoid infeasibility

# Metrics & helpers

def smape(y_true, y_pred, eps=1e-9) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(eps, (np.abs(y_true) + np.abs(y_pred)) / 2.0)
    return float(np.mean(np.abs(y_true - y_pred) / denom))


def add_calendar_features(df: pd.DataFrame, min_date: pd.Timestamp) -> pd.DataFrame:
    out = df.copy()
    out["date"] = pd.to_datetime(out["date"])
    out["dow"] = out["date"].dt.dayofweek
    out["dayofyear"] = out["date"].dt.dayofyear
    out["sin_doy"] = np.sin(2 * np.pi * out["dayofyear"] / 365.25)
    out["cos_doy"] = np.cos(2 * np.pi * out["dayofyear"] / 365.25)
    out["t"] = (out["date"] - min_date).dt.days
    out["is_weekend"] = out["dow"].isin([5, 6]).astype(int)
    return out


def add_lag_rolling_features(panel: pd.DataFrame) -> pd.DataFrame:
    """
    Panel must be sorted by (product_id, date).
    Creates lag features and rolling stats on sales.
    Uses shift(1) so today's features don't leak today's target.
    """
    df = panel.copy()
    df = df.sort_values(["product_id", "date"])

    for lag in LAGS:
        df[f"lag_{lag}"] = df.groupby("product_id")["sales"].shift(lag)

    for w in ROLL_WINDOWS:
        s = df.groupby("product_id")["sales"].shift(1)
        df[f"roll_mean_{w}"] = s.groupby(df["product_id"]).rolling(w).mean().reset_index(level=0, drop=True)
        df[f"roll_std_{w}"] = s.groupby(df["product_id"]).rolling(w).std().reset_index(level=0, drop=True)

    return df


def candidate_prices(prod_df: pd.DataFrame, buy_last: float, last_price: float) -> np.ndarray:
    prices = np.sort(np.unique(np.round(prod_df["sell_price"].astype(float).values, 2)))
    prices = prices[prices > buy_last + 1e-2]  # avoid negative unit profit candidates

    if len(prices) == 0:
        prices = np.array([round(buy_last + 0.01, 2)])

    max_hist = float(prices.max())
    uplift = np.linspace(max_hist, max_hist * PRICE_UPLIFT_CAP, 10)
    prices = np.unique(np.round(np.concatenate([prices, uplift, [last_price]]), 2))

    if len(prices) > MAX_PRICE_CANDIDATES:
        qs = np.linspace(0, 1, MAX_PRICE_CANDIDATES)
        prices = np.unique(np.round(np.quantile(prices, qs), 2))

    return np.sort(prices)



# Objective 2 — Forecasting (3 approaches)

FORECAST_FEATURES = [
    "sell_price",
    "t", "sin_doy", "cos_doy",
    "dow", "is_weekend",
    # lags/rolling
    *[f"lag_{l}" for l in LAGS],
    *[f"roll_mean_{w}" for w in ROLL_WINDOWS],
    *[f"roll_std_{w}" for w in ROLL_WINDOWS],
    # product id (categorical)
    "product_id",
]

TARGET_COL = "sales"


@dataclass
class Obj2Models:
    cat: CatBoostRegressor
    lgbm: lgb.LGBMRegressor
    min_date: pd.Timestamp
    dow_means: dict  # per product_id: Series indexed by dow


def make_obj2_train_val(panel: pd.DataFrame, val_days: int = 30):
    """
    Time split: last `val_days` points per product are validation.
    """
    df = panel.copy().sort_values(["product_id", "date"])
    df["rank_desc"] = df.groupby("product_id")["date"].rank(method="first", ascending=False)
    val = df[df["rank_desc"] <= val_days].copy()
    trn = df[df["rank_desc"] > val_days].copy()
    trn.drop(columns=["rank_desc"], inplace=True)
    val.drop(columns=["rank_desc"], inplace=True)
    return trn, val


def fit_obj2_models(panel: pd.DataFrame) -> Obj2Models:
    """
    Fit two strong global models:
      - CatBoostRegressor (handles non-linearities, categorical)
      - LightGBM Quantile Regressor (median; robust; probabilistic-aware)
    Baseline model (dow_mean) is computed separately.
    """
    min_date = panel["date"].min()
    df = add_calendar_features(panel, min_date)
    df = add_lag_rolling_features(df)

    # Drop rows with missing lag/rolling features (early history)
    df_ml = df.dropna(subset=[c for c in FORECAST_FEATURES if c != "product_id"]).copy()

    # Prepare baseline: weekday means per product
    df_ml["dow"] = df_ml["dow"].astype(int)
    dow_means = {
        pid: grp.groupby("dow")["sales"].mean()
        for pid, grp in df_ml.groupby("product_id")
    }

    X = df_ml[FORECAST_FEATURES].copy()
    y = np.log1p(df_ml[TARGET_COL].values)

    # CatBoost (global) — very strong for retail
    X_cat = X.copy()
    X_cat["product_id"] = X_cat["product_id"].astype(str)
    cat_features = [X_cat.columns.get_loc("product_id")]  # treat product_id as categorical

    cat = CatBoostRegressor(
        iterations=800,
        depth=6,
        learning_rate=0.05,
        loss_function="RMSE",
        random_seed=0,
        verbose=False
    )
    cat.fit(X_cat, y, cat_features=cat_features)

    # LightGBM Quantile (median) — robust and "latest-market" probabilistic angle
    X_lgb = X.copy()
    X_lgb["product_id"] = X_lgb["product_id"].astype("category")

    lgbm = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.5,  # median
        n_estimators=1200,
        learning_rate=0.03,
        num_leaves=64,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=0
    )
    lgbm.fit(X_lgb, y)

    return Obj2Models(cat=cat, lgbm=lgbm, min_date=min_date, dow_means=dow_means)


def evaluate_obj2_approaches(panel: pd.DataFrame, models: Obj2Models, val_days: int = 30):
    """
    Evaluate 3 approaches on time-holdout:
      A) weekday mean baseline
      B) CatBoost global model
      C) LightGBM quantile global model

    Returns: per-product sMAPE table and chosen approach per product.
    """
    trn, val = make_obj2_train_val(panel, val_days=val_days)

    # Build features for validation
    val_f = add_calendar_features(val, models.min_date)
    val_f = add_lag_rolling_features(val_f)
    val_f = val_f.dropna(subset=[c for c in FORECAST_FEATURES if c != "product_id"]).copy()

    y_true = val_f["sales"].values

    # A) Baseline: weekday mean
    pred_base = []
    for pid, dow in zip(val_f["product_id"], val_f["dow"]):
        series = models.dow_means.get(pid)
        if series is None:
            pred_base.append(float(panel[panel["product_id"] == pid]["sales"].mean()))
        else:
            pred_base.append(float(series.get(int(dow), float(series.mean()))))
    pred_base = np.array(pred_base)

    # B) CatBoost
    X_cat = val_f[FORECAST_FEATURES].copy()
    X_cat["product_id"] = X_cat["product_id"].astype(str)
    pred_cat = np.expm1(models.cat.predict(X_cat)).clip(0, None)

    # C) LightGBM
    X_lgb = val_f[FORECAST_FEATURES].copy()
    X_lgb["product_id"] = X_lgb["product_id"].astype("category")
    pred_lgb = np.expm1(models.lgbm.predict(X_lgb)).clip(0, None)

    # Compute per-product sMAPE
    rows = []
    for pid in sorted(val_f["product_id"].unique()):
        mask = (val_f["product_id"] == pid).values
        rows.append({
            "product_id": pid,
            "smape_baseline_dow": smape(y_true[mask], pred_base[mask]),
            "smape_catboost": smape(y_true[mask], pred_cat[mask]),
            "smape_lightgbm_q50": smape(y_true[mask], pred_lgb[mask]),
        })
    score_df = pd.DataFrame(rows)

    # choose best per product
    def choose(row):
        opts = {
            "baseline_dow": row["smape_baseline_dow"],
            "catboost": row["smape_catboost"],
            "lightgbm_q50": row["smape_lightgbm_q50"],
        }
        return min(opts, key=opts.get)

    score_df["chosen_forecaster"] = score_df.apply(choose, axis=1)
    return score_df


def _forecast_recursive_one_product(
    hist: pd.DataFrame,
    pid,
    future_dates,
    fixed_price: float,
    models: Obj2Models,
    chosen: str
) -> np.ndarray:
    """
    Recursive forecasting for ML models because lag features depend on past sales.
    For baseline, no recursion needed.
    """
    hist = hist[hist["product_id"] == pid].sort_values("date").copy()
    min_date = models.min_date

    # Baseline (weekday mean) doesn't need recursion
    if chosen == "baseline_dow":
        series = models.dow_means.get(pid)
        if series is None:
            mu = float(hist["sales"].mean())
            return np.full(len(future_dates), mu)
        preds = []
        for dt in pd.to_datetime(future_dates):
            preds.append(float(series.get(int(dt.dayofweek), float(series.mean()))))
        return np.array(preds)

    # For ML models, iterate day by day and append predicted sales
    work = hist[["product_id", "date", "sell_price", "sales"]].copy()
    preds = []

    for dt in pd.to_datetime(future_dates):
        # append "future row" with unknown sales, then compute features on the fly
        tmp = pd.DataFrame([{
            "product_id": pid,
            "date": dt,
            "sell_price": fixed_price,
            "sales": np.nan
        }])
        work2 = pd.concat([work, tmp], ignore_index=True).sort_values("date")

        feat = add_calendar_features(work2, min_date)
        feat = add_lag_rolling_features(feat)
        row = feat.iloc[-1:].copy()

        # Fill any remaining NaNs (should be rare with enough history)
        for c in [f"lag_{l}" for l in LAGS]:
            if row[c].isna().any():
                row[c] = work["sales"].mean()
        for c in [f"roll_mean_{w}" for w in ROLL_WINDOWS] + [f"roll_std_{w}" for w in ROLL_WINDOWS]:
            if row[c].isna().any():
                row[c] = work["sales"].mean()

        if chosen == "catboost":
            X = row[FORECAST_FEATURES].copy()
            X["product_id"] = X["product_id"].astype(str)
            yhat = float(np.expm1(models.cat.predict(X))[0])
        elif chosen == "lightgbm_q50":
            X = row[FORECAST_FEATURES].copy()
            X["product_id"] = X["product_id"].astype("category")
            yhat = float(np.expm1(models.lgbm.predict(X))[0])
        else:
            raise ValueError(f"Unknown Obj2 model: {chosen}")

        yhat = max(0.0, yhat)
        preds.append(yhat)

        # now append predicted sales to history and continue
        work = pd.concat([work, pd.DataFrame([{
            "product_id": pid,
            "date": dt,
            "sell_price": fixed_price,
            "sales": yhat
        }])], ignore_index=True)

    return np.array(preds)

# Demand models for pricing (3 approaches)

DEMAND_FEATURES = ["sell_price", "t", "sin_doy", "cos_doy", "dow", "is_weekend"]


@dataclass
class DemandModelBundle:
    best_name: str
    ridge: Ridge
    lgbm: lgb.LGBMRegressor
    cat: CatBoostRegressor
    min_date: pd.Timestamp


def fit_demand_models_per_product(dprod: pd.DataFrame) -> DemandModelBundle:
    """
    Fit 3 demand models for a single product:
      1) Log-log Ridge (interpretable elasticity-ish)
      2) LightGBM with monotonic constraint on sell_price (demand decreases as price increases)
      3) CatBoost with monotonic constraint on sell_price (same idea)

    Selection is done via time holdout sMAPE.
    """
    dprod = dprod.sort_values("date").copy()
    min_date = dprod["date"].min()

    df = add_calendar_features(dprod, min_date)
    X = df[DEMAND_FEATURES].copy()
    y = np.log1p(df["sales"].values)

    # time split
    n = len(df)
    val_n = min(30, max(7, int(0.2 * n)))
    trn = df.iloc[:-val_n].copy()
    val = df.iloc[-val_n:].copy()

    X_trn = trn[DEMAND_FEATURES].copy()
    y_trn = np.log1p(trn["sales"].values)
    X_val = val[DEMAND_FEATURES].copy()
    y_val_true = val["sales"].values

    # 1) log-log ridge: add log_price explicitly
    Xr_trn = X_trn.copy()
    Xr_val = X_val.copy()
    Xr_trn["log_price"] = np.log(Xr_trn["sell_price"].astype(float).clip(1e-6))
    Xr_val["log_price"] = np.log(Xr_val["sell_price"].astype(float).clip(1e-6))
    ridge_cols = ["log_price", "t", "sin_doy", "cos_doy", "dow", "is_weekend"]
    ridge = Ridge(alpha=1.0, random_state=0)
    ridge.fit(Xr_trn[ridge_cols], y_trn)
    pred_ridge = np.expm1(ridge.predict(Xr_val[ridge_cols])).clip(0, None)

    # 2) LightGBM monotonic on sell_price (feature index 0)
    lgbm = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1500,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=0,
        monotone_constraints=[-1, 0, 0, 0, 0, 0],  # sell_price decreasing
    )
    lgbm.fit(X_trn, y_trn)
    pred_lgb = np.expm1(lgbm.predict(X_val)).clip(0, None)

    # 3) CatBoost monotonic on sell_price
    # monotone list aligns with DEMAND_FEATURES
    cat = CatBoostRegressor(
        iterations=1200,
        depth=6,
        learning_rate=0.03,
        loss_function="RMSE",
        random_seed=0,
        verbose=False,
        monotone_constraints=[-1, 0, 0, 0, 0, 0],
    )
    cat.fit(X_trn, y_trn)
    pred_cat = np.expm1(cat.predict(X_val)).clip(0, None)

    scores = {
        "ridge_loglog": smape(y_val_true, pred_ridge),
        "lightgbm_mono": smape(y_val_true, pred_lgb),
        "catboost_mono": smape(y_val_true, pred_cat),
    }
    best_name = min(scores, key=scores.get)

    # Refit all on full data for final use
    ridge.fit(
        pd.concat([Xr_trn, Xr_val])[ridge_cols],
        np.log1p(pd.concat([trn, val])["sales"].values)
    )
    lgbm.fit(X, y)
    cat.fit(X, y)

    return DemandModelBundle(best_name=best_name, ridge=ridge, lgbm=lgbm, cat=cat, min_date=min_date)


def demand_predict(bundle: DemandModelBundle, dates, prices) -> np.ndarray:
    tmp = pd.DataFrame({"date": pd.to_datetime(dates), "sell_price": np.asarray(prices, dtype=float)})
    tmp = add_calendar_features(tmp, bundle.min_date)
    X = tmp[DEMAND_FEATURES].copy()

    if bundle.best_name == "ridge_loglog":
        Xr = X.copy()
        Xr["log_price"] = np.log(Xr["sell_price"].astype(float).clip(1e-6))
        ridge_cols = ["log_price", "t", "sin_doy", "cos_doy", "dow", "is_weekend"]
        return np.expm1(bundle.ridge.predict(Xr[ridge_cols])).clip(0, None)

    if bundle.best_name == "lightgbm_mono":
        return np.expm1(bundle.lgbm.predict(X)).clip(0, None)

    if bundle.best_name == "catboost_mono":
        return np.expm1(bundle.cat.predict(X)).clip(0, None)

    raise ValueError(f"Unknown demand model: {bundle.best_name}")


# =============================
# Pricing optimization (3 approaches)
# =============================

def build_items(df: pd.DataFrame, future_dates, demand_fn):
    last = df.sort_values("date").groupby("product_id").tail(1).set_index("product_id")
    items = []
    for pid, dprod in df.groupby("product_id"):
        dprod = dprod.sort_values("date")
        buy_last = float(last.loc[pid, "buy_price"])
        last_price = float(last.loc[pid, "sell_price"])
        cand = candidate_prices(dprod, buy_last, last_price)

        for dt in future_dates:
            sales_hat = demand_fn(pid, [dt] * len(cand), cand)
            revenue = cand * sales_hat
            profit = (cand - buy_last) * sales_hat
            items.append({
                "pid": pid,
                "date": dt,
                "buy_last": buy_last,
                "cand": cand,
                "sales_hat": sales_hat,
                "rev": revenue,
                "prof": profit,
            })
    return items


def pick_by_lambda(items, lmbda: float):
    rows, tot_rev, tot_prof = [], 0.0, 0.0
    for it in items:
        score = it["rev"] + lmbda * it["prof"]
        j = int(np.argmax(score))
        rows.append({
            "product_id": it["pid"],
            "date": it["date"].date(),
            "sell_price": round(float(it["cand"][j]), 2),
            "pred_sales": float(it["sales_hat"][j]),
            "expected_revenue": float(it["rev"][j]),
            "expected_profit": float(it["prof"][j]),
        })
        tot_rev += float(it["rev"][j])
        tot_prof += float(it["prof"][j])
    return pd.DataFrame(rows), tot_rev, tot_prof


def optimize_lagrangian(items, profit_min_total: float):
    # feasibility
    df_hi, rev_hi, prof_hi = pick_by_lambda(items, 1e6)
    if prof_hi + 1e-9 < profit_min_total:
        return None, None, None, prof_hi

    lo, hi = 0.0, 1e6
    best_df, best_rev, best_prof = None, -np.inf, -np.inf

    for _ in range(50):
        mid = (lo + hi) / 2
        df_mid, rev_mid, prof_mid = pick_by_lambda(items, mid)
        if prof_mid >= profit_min_total:
            best_df, best_rev, best_prof = df_mid, rev_mid, prof_mid
            hi = mid
        else:
            lo = mid

    return best_df, best_rev, best_prof, prof_hi


def optimize_greedy_repair(items, profit_min_total: float):
    base_df, base_rev, base_prof = pick_by_lambda(items, 0.0)
    if base_prof >= profit_min_total:
        return base_df, base_rev, base_prof

    curr_idx = np.array([int(np.argmax(it["rev"])) for it in items], dtype=int)
    cur_rev = float(sum(it["rev"][j] for it, j in zip(items, curr_idx)))
    cur_prof = float(sum(it["prof"][j] for it, j in zip(items, curr_idx)))

    for _ in range(6000):
        if cur_prof >= profit_min_total:
            break

        best_move, best_score = None, -np.inf
        for k, it in enumerate(items):
            j0 = curr_idx[k]
            for j1 in range(j0 + 1, len(it["cand"])):
                d_prof = float(it["prof"][j1] - it["prof"][j0])
                if d_prof <= 1e-9:
                    continue
                d_rev = float(it["rev"][j1] - it["rev"][j0])
                score = (1e12 + d_prof) if d_rev >= 0 else (d_prof / (-d_rev + 1e-9))
                if score > best_score:
                    best_score, best_move = score, (k, j1, d_rev, d_prof)

        if best_move is None:
            break

        k, j1, d_rev, d_prof = best_move
        curr_idx[k] = j1
        cur_rev += d_rev
        cur_prof += d_prof

    if cur_prof < profit_min_total:
        return None, None, None

    rows = []
    for it, j in zip(items, curr_idx):
        rows.append({
            "product_id": it["pid"],
            "date": it["date"].date(),
            "sell_price": round(float(it["cand"][j]), 2),
            "pred_sales": float(it["sales_hat"][j]),
            "expected_revenue": float(it["rev"][j]),
            "expected_profit": float(it["prof"][j]),
        })
    out = pd.DataFrame(rows)
    return out, float(out["expected_revenue"].sum()), float(out["expected_profit"].sum())


def optimize_constant_per_product(df, future_dates, profit_min_total, demand_fn):
    """
    Interpretable approach: pick ONE constant price per product over the horizon.
    We still use the selected demand model to compute demand at each candidate price.
    """
    last = df.sort_values("date").groupby("product_id").tail(1).set_index("product_id")
    choices = []

    for pid, dprod in df.groupby("product_id"):
        dprod = dprod.sort_values("date")
        buy_last = float(last.loc[pid, "buy_price"])
        last_price = float(last.loc[pid, "sell_price"])
        cand = candidate_prices(dprod, buy_last, last_price)

        best = None
        for p in cand:
            s = demand_fn(pid, future_dates, [p] * len(future_dates))
            rev = float(np.sum(s * p))
            prof = float(np.sum(s * (p - buy_last)))
            if best is None or rev > best[1]:
                best = (p, rev, prof)

        choices.append((pid, best[0]))

    # Expand to daily plan and evaluate constraint
    rows = []
    tot_rev, tot_prof = 0.0, 0.0
    for pid, p in choices:
        buy_last = float(last.loc[pid, "buy_price"])
        s = demand_fn(pid, future_dates, [p] * len(future_dates))
        for dt, q in zip(future_dates, s):
            rev = float(q * p)
            prof = float(q * (p - buy_last))
            rows.append({
                "product_id": pid,
                "date": dt.date(),
                "sell_price": round(float(p), 2),
                "pred_sales": float(q),
                "expected_revenue": rev,
                "expected_profit": prof,
            })
            tot_rev += rev
            tot_prof += prof

    if tot_prof < profit_min_total:
        return None, None, None
    out = pd.DataFrame(rows)
    return out, tot_rev, tot_prof


# =============================
# Main
# =============================
def main():
    df = pd.read_csv(DATA_PATH)
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(["product_id", "date"]).copy()

    # infer buy_price from margin definition
    df["buy_price"] = df["sell_price"] - df["margin"]

    last_date = df["date"].max()
    future_dates = [last_date + timedelta(days=i) for i in range(1, HORIZON_DAYS + 1)]

    last = df.groupby("product_id").tail(1).set_index("product_id")
    profit_last_day_total = float((last["margin"] * last["sales"]).sum())
    profit_min_total = 7.0 * profit_last_day_total

    print(f"Last date: {last_date.date()}")
    print(f"Profit last day (total): {profit_last_day_total:.2f}")
    print(f"Profit constraint (7-day) >= {profit_min_total:.2f}")

    # -----------------------------
    # Objective 2: forecasting
    # -----------------------------
    obj2_models = fit_obj2_models(df)
    obj2_scores = evaluate_obj2_approaches(df, obj2_models, val_days=30)

    # Forecast next 7 days per product using chosen model per product
    forecast_rows = []
    for pid in df["product_id"].unique():
        chosen = obj2_scores.loc[obj2_scores["product_id"] == pid, "chosen_forecaster"].iloc[0]
        last_price = float(last.loc[pid, "sell_price"])
        buy_last = float(last.loc[pid, "buy_price"])

        sales_hat = _forecast_recursive_one_product(
            hist=df,
            pid=pid,
            future_dates=future_dates,
            fixed_price=last_price,
            models=obj2_models,
            chosen=chosen
        )

        for dt, s in zip(future_dates, sales_hat):
            forecast_rows.append({
                "product_id": pid,
                "date": dt.date(),
                "sell_price": round(last_price, 2),
                "pred_sales": float(s),
                "expected_revenue": float(s * last_price),
                "expected_profit": float(s * (last_price - buy_last)),
                "model_used": chosen
            })

    forecast_df = pd.DataFrame(forecast_rows)
    forecast_df.to_csv("objective2_forecast.csv", index=False)
    print("Saved -> objective2_forecast.csv")

    # Objective 1: demand models (per product) for pricing

    demand_bundles = {}
    demand_selection_rows = []

    for pid, dprod in df.groupby("product_id"):
        bundle = fit_demand_models_per_product(dprod)
        demand_bundles[pid] = bundle
        demand_selection_rows.append({
            "product_id": pid,
            "objective": "obj1_demand_model",
            "chosen_model": bundle.best_name,
        })

    # demand fn for optimization
    def demand_fn(pid, dates, prices):
        return demand_predict(demand_bundles[pid], dates, prices)

    # Build discrete decision items
    items = build_items(df, future_dates, demand_fn)

    # Pricing optimizers
    results = []

    df_a1, rev_a1, prof_a1 = optimize_constant_per_product(df, future_dates, profit_min_total, demand_fn)
    if df_a1 is not None:
        results.append(("A1_constant_per_product", df_a1, rev_a1, prof_a1))

    df_a2, rev_a2, prof_a2 = optimize_greedy_repair(items, profit_min_total)
    if df_a2 is not None:
        results.append(("A2_greedy_repair", df_a2, rev_a2, prof_a2))

    df_a3, rev_a3, prof_a3, prof_max = optimize_lagrangian(items, profit_min_total)
    if df_a3 is not None:
        results.append(("A3_lagrangian", df_a3, rev_a3, prof_a3))

    if not results:
        raise ValueError(
            "No feasible pricing solution found with current candidate grid. "
            "Increase PRICE_UPLIFT_CAP or enrich demand features."
        )

    best_name, best_df, best_rev, best_prof = sorted(results, key=lambda x: x[2], reverse=True)[0]
    best_df = best_df.copy()
    best_df["approach"] = best_name
    best_df.to_csv("objective1_optimal_pricing.csv", index=False)
    print("Saved -> objective1_optimal_pricing.csv")
    print(f"Chosen pricing approach: {best_name}")
    print(f"Objective 1 totals: revenue={best_rev:.2f}, profit={best_prof:.2f} (min={profit_min_total:.2f})")

    # save model selection info
    model_selection = obj2_scores.copy()
    model_selection["objective"] = "obj2_forecasting"
    model_selection = pd.concat([model_selection, pd.DataFrame(demand_selection_rows)], ignore_index=True)
    model_selection.to_csv("model_selection.csv", index=False)
    print("Saved -> model_selection.csv")

    # save approach comparison
    comp = []
    for name, sol, r, p in results:
        comp.append({
            "approach": name,
            "total_expected_revenue": float(r),
            "total_expected_profit": float(p),
            "profit_constraint": float(profit_min_total),
            "feasible": bool(p >= profit_min_total),
        })
    pd.DataFrame(comp).to_csv("approach_comparison.csv", index=False)
    print("Saved -> approach_comparison.csv")


if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit